In [1]:
import torch
from torch import nn


class CircleModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=10)
        self.layer_2 = nn.Linear(in_features=10, out_features=10)
        self.layer_3 = nn.Linear(in_features=10, out_features=1)

    def forward(self, x):
        z = self.layer_1(x)
        z = self.layer_2(z)
        z = self.layer_3(z)

        # z = self.layer_3(self.layer_2(self.layer_1(x)))
        return z
    
model_v2 = CircleModelV2()
model_v2.to("cuda" if torch.cuda.is_available() else "cpu")

CircleModelV2(
  (layer_1): Linear(in_features=2, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=1, bias=True)
)

In [2]:
loss_fun = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_v2.parameters(),
                            lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

X_train, y_train = X_train.to("cuda" if torch.cuda.is_available() else "cpu"), y_train.to("cuda" if torch.cuda.is_available() else "cpu")
X_test, y_test = X_test.to("cuda" if torch.cuda.is_available() else "cpu"), y_test.to("cuda" if torch.cuda.is_available() else "cpu")

epochs = 1000
for epoch in range(epochs):
    model_v2.train()
    y_logits = model_v2(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))


    loss = loss_fun(y_logits, y_train)
    acc = (y_pred == y_train).sum() / len(y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step() # Adjust the model parameters

    model_v2.eval()
    with torch.inference_mode():
        test_logits = model_v2(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fun(test_logits, y_test)
        test_acc = (test_pred == y_test).sum() / len(test_pred)
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | "
              f"Train Loss: {loss:.5f} | Train Acc: {acc:.2f} | "
              f"Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}")

